<a href="https://colab.research.google.com/github/tomek-l/video-mining/blob/master/cifar10_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision
from torchvision import models
from torchvision import transforms

In [0]:
import numpy as np
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Move channel axes - go from (N,H,W,C) convention to (N,C,H,W)
X_train = np.moveaxis(X_train, 3, 1)
X_test = np.moveaxis(X_test, 3, 1)

# Convert Feature Vectors to FP32
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

y_train = y_train.astype(np.int)
y_test = y_test.astype(np.int)

y_train = np.squeeze(y_train)
y_test = np.squeeze(y_test)

In [0]:
from torch.utils.data.dataset import Dataset
 
class CIFAR10_custom_ds(Dataset):
    def __init__(self, feature_vec, labels, device):
        self._device = device
        self.x = feature_vec
        self.y = labels 

        # I only do eager conversion to CUDA memory since CIFAR10 is relatively small
        # (will only take some ~200MB of memory)
        self.x = torch.tensor(self.x).to(self._device)
        self.y = torch.tensor(self.y).to(self._device)
         
    def __getitem__(self, index):
        return (self.x[index], self.y[index])
 
    def __len__(self):
        return len(self.x)

In [0]:
# pytorch way to use CIFAR-10
# (with DataLoader class)

# transform = transforms.Compose(
#     [transforms.ToTensor(),
#      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
#                                         download=True, transform=transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
#                                           shuffle=True, num_workers=2)

# testset = torchvision.datasets.CIFAR10(root='./data', train=False,
#                                        download=True, transform=transform)
# testloader = torch.utils.data.DataLoader(testset, batch_size=4,
#                                          shuffle=False, num_workers=2)

# classes = ('plane', 'car', 'bird', 'cat',
#            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [0]:
import torch.nn as nn

# Let's define a simple NN based of resnet18
class Net(nn.Module):
    def __init__(self, n_classes):
        super(Net, self).__init__()
        self.backbone = models.resnet18(pretrained=False)
        self.backbone.fc = torch.nn.Linear(512, n_classes)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.backbone(x)
        x = self.sigmoid(x)
        return x


In [0]:
x,y = trainset[0]

In [0]:
from torch.utils.data import DataLoader

# import torch.multiprocessing as mp
# try:
#     mp.set_start_method("spawn")
# except RuntimeError as e:
#     print(e)
#     pass

GPU = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
trainset = CIFAR10_custom_ds(X_train, y_train, GPU)
assert trainset[0:4] 
trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=0)

In [84]:
labels.shape

torch.Size([16])

In [92]:
from torch import optim 

net = Net(10).to(GPU)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
print_every = 100
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % print_every == 1:    # print every 100 mini-batches
            print(f'epoch {epoch+1} batch {i+1} loss: {(running_loss / print_every):.4f}')
            running_loss = 0.0

print('Finished Training')

epoch 1 iter 1 loss: 0.0233
epoch 1 iter 101 loss: 2.2289
epoch 1 iter 201 loss: 2.1024
epoch 1 iter 301 loss: 2.0453
epoch 1 iter 401 loss: 2.0010
epoch 1 iter 501 loss: 1.9711
epoch 1 iter 601 loss: 1.9552
epoch 1 iter 701 loss: 1.9431
epoch 2 iter 1 loss: 0.0188
epoch 2 iter 101 loss: 1.9089
epoch 2 iter 201 loss: 1.9049
epoch 2 iter 301 loss: 1.8962
epoch 2 iter 401 loss: 1.8930
epoch 2 iter 501 loss: 1.8845
epoch 2 iter 601 loss: 1.8800
epoch 2 iter 701 loss: 1.8789
epoch 3 iter 1 loss: 0.0188
epoch 3 iter 101 loss: 1.8494
epoch 3 iter 201 loss: 1.8501
epoch 3 iter 301 loss: 1.8524
epoch 3 iter 401 loss: 1.8436
epoch 3 iter 501 loss: 1.8422
epoch 3 iter 601 loss: 1.8365
epoch 3 iter 701 loss: 1.8385
epoch 4 iter 1 loss: 0.0183
epoch 4 iter 101 loss: 1.8253
epoch 4 iter 201 loss: 1.8155
epoch 4 iter 301 loss: 1.8165
epoch 4 iter 401 loss: 1.8071
epoch 4 iter 501 loss: 1.8108
epoch 4 iter 601 loss: 1.8075
epoch 4 iter 701 loss: 1.8075
epoch 5 iter 1 loss: 0.0171
epoch 5 iter 101 los

In [93]:
testset = CIFAR10_custom_ds(X_test, y_test, GPU)
testloader = DataLoader(testset, batch_size=64, shuffle=True, num_workers=0)

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the  {len(y_test)} test images is {100 * correct / total} %')


Accuracy of the network on the  10000 test images is 60.81 %
